In [1]:
import os
import re
import random

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn import model_selection
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import scipy.linalg

In [16]:
def list_files(dir):
    folders = []
    for root, dirs, files in os.walk(dir):
        for r in dirs:
            folders.append(r)
    return folders

def list_directories(folders_list):
    images_dirs = []
    for i in range(len(folders)):
        images_d = '/Users/jingyany/Desktop/17Spring/Data558/data competation/train/'+folders[i]+'/'
        images_dirs.append(images_d)
    return images_dirs

def images_lists(images_dirs):
    list_images = []
    for i in range(len(images_dirs)):
        images_dir = images_dirs[i]
        for f in os.listdir(images_dir):
            if re.search('jpg|JPG', f):
                list_images.append(images_dir + f)
    return list_images

In [4]:
model_dir = '/Users/jingyany/models/tutorials/image/imagenet/TUTORIAL_DIR/imagenet'
dir = '/Users/jingyany/Desktop/17Spring/Data558/data competation/train'
folders = list_files(dir)
images_dirs = list_directories(folders)
list_images = images_lists(images_dirs)

In [5]:
def create_graph():
    with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
        
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        #if (ind%100 == 0):
        print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        features[ind,:] = np.squeeze(predictions)
        labels.append(re.split('_\d+',image.split('/')[-2].split('.')[0])[0])
    labels = list(map(int, labels))

    return features, labels

In [6]:
features,labels = extract_features(list_images)

Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0003_796136.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0023_796059.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0024_796089.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057.jpg...


In [11]:
labels = np.asarray(labels)
y_train = labels
X_train = features

In [30]:
model_dir = '/Users/jingyany/models/tutorials/image/imagenet/TUTORIAL_DIR/imagenet'
dir_test = '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/'
list_images_test = [dir_test+f for f in os.listdir(dir_test) if re.search('jpg|JPG', f)]
list_images_test

['/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/10.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/100.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1000.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1001.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1002.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1003.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1004.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1005.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1006.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1007.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1008.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1009.jpg',
 '/Users/jingyany/

In [31]:
def extract_features_test(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    #labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        #if (ind%100 == 0):
        print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        features[ind,:] = np.squeeze(predictions)
        #labels.append(re.split('_\d+',image.split('/')[-2].split('.')[0])[0])
    #labels = list(map(int, labels))

    return features

In [32]:
features_test = extract_features_test(list_images_test)

Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/10.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/100.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1000.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1001.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1002.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1003.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1004.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1005.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1006.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/test/1007.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data com

In [34]:
features_test.shape

(4320, 2048)